# RoBERTa Classifier (no pretraining)

In this notebook we will train a RoBERTa classifier from scratch for the proxy task.  This notebook is adapted from [this](https://towardsdatascience.com/fastai-with-transformers-bert-roberta-xlnet-xlm-distilbert-4f41ee18ecb2) blog post.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from fastai import *
from fastai.text import *
from transformers import RobertaForSequenceClassification, RobertaConfig
import eval_models
from train_utils import *

In [ ]:
import fastai
import transformers
import tokenizers
print('fastai version :', fastai.__version__)
print('transformers version :', transformers.__version__)
print('tokenizers version :', tokenizers.__version__)

In [ ]:
torch.cuda.set_device(0)

### Prep Databunch

In [ ]:
bs = 64
seed = 42
tok_model_dir = '/home/dyang/InstrumentID/tokenizer/roberta_tok/shift0'
max_seq_len = 256

In [ ]:
cust_tok = CustomTokenizer(TransformersBaseTokenizer, tok_model_dir, max_seq_len)
transformer_base_tokenizer = TransformersBaseTokenizer(tok_model_dir, max_seq_len)
transformer_vocab =  TransformersVocab(tokenizer = transformer_base_tokenizer._pretrained_tokenizer)

In [ ]:
pad_idx = transformer_base_tokenizer._pretrained_tokenizer.token_to_id('<pad>')

In [ ]:
bpe_path = Path('/home/dyang/InstrumentID/train_data')
train_df = pd.read_csv(bpe_path/'train_df-frag64.char.csv')
valid_df = pd.read_csv(bpe_path/'valid_df-frag64.char.csv')
test_df = pd.read_csv(bpe_path/'test_df-frag64.char.csv')

In [ ]:
data_clas = TextDataBunch.from_df(bpe_path, train_df, valid_df, tokenizer=cust_tok, vocab=transformer_vocab,
                                  include_bos=False, include_eos=False, pad_first=False, pad_idx=pad_idx, 
                                  bs=bs, num_workers=1)

### Train Classifier

In [ ]:
model_class, config_class = RobertaForSequenceClassification, RobertaConfig
model_path = '/home/dyang/InstrumentID/roBERTa_train-target/'

In [ ]:
config = config_class.from_pretrained(model_path)
config.num_labels = data_clas.c

In [ ]:
transformer_model = model_class(config) # not pretrained
custom_transformer_model = RobertaModelWrapper(transformer_model, pad_idx)

In [ ]:
# learner.destroy()
# torch.cuda.empty_cache()

In [ ]:
learner = Learner(data_clas, custom_transformer_model, metrics=[accuracy, FBeta(average='macro', beta=1)])

In [ ]:
list_layers = [learner.model.transformer.roberta.embeddings,
              learner.model.transformer.roberta.encoder.layer[0],
              learner.model.transformer.roberta.encoder.layer[1],
              learner.model.transformer.roberta.encoder.layer[2],
              learner.model.transformer.roberta.encoder.layer[3],
              learner.model.transformer.roberta.encoder.layer[4],
              learner.model.transformer.roberta.encoder.layer[5],
              learner.model.transformer.roberta.pooler]

In [ ]:
learner.split(list_layers)
print(learner.layer_groups)

In [ ]:
seed_all(seed)

In [ ]:
learner.lr_find()

In [ ]:
learner.recorder.plot(suggestion=True)

In [ ]:
lr = 3e-5

In [ ]:
learner.unfreeze()

In [ ]:
learner.fit_one_cycle(2, lr, moms=(0.8,0.7))

In [ ]:
learner.fit_one_cycle(4, lr, moms=(0.8,0.7))

In [ ]:
learner.save('roberta_clas')
#learner.load('roberta_clas')

### Evaluate Classifier

Evaluate on the proxy task -- classifying fixed-length chunks of bootleg score features.

In [ ]:
data_clas_test = TextDataBunch.from_df(bpe_path, train_df, test_df, tokenizer=cust_tok, vocab=transformer_vocab,
                                  include_bos=False, include_eos=False, pad_first=False, pad_idx=pad_idx, 
                                  bs=bs, num_workers=1)

In [ ]:
learner.validate(data_clas_test.valid_dl)

Evaluate on the original task -- classifying pages of sheet music.  We can evaluate our models in two ways:
- applying the model to a variable length sequence
- applying the model to multiple fixed-length windows and averaging the predictions

First we evaluate the model on variable length inputs.  Report results with and without applying priors.

In [ ]:
train_fullpage_df = pd.read_csv(bpe_path/'fullpage_train.char.csv')
valid_fullpage_df = pd.read_csv(bpe_path/'fullpage_valid.char.csv')
test_fullpage_df = pd.read_csv(bpe_path/'fullpage_test.char.csv')

In [ ]:
data_clas_test = TextDataBunch.from_df(bpe_path, train_fullpage_df, valid_fullpage_df, test_fullpage_df,
                                       tokenizer=cust_tok, vocab=transformer_vocab, include_bos=False, 
                                       include_eos=False, pad_first=False, pad_idx=pad_idx, bs=bs, num_workers=1)

In [ ]:
(acc, acc_with_prior), (f1, f1_with_prior) = eval_models.calcAccuracy_fullpage(learner, bpe_path, train_fullpage_df, valid_fullpage_df, test_fullpage_df, databunch=data_clas_test)
(acc, acc_with_prior), (f1, f1_with_prior)

Now we evaluate the model by considering multiple fixed-length windows and averaging the predictions.

In [ ]:
test_ensemble_df = pd.read_csv(bpe_path/'test.ensemble64.char.csv')

In [ ]:
data_clas_test = TextDataBunch.from_df(bpe_path, train_fullpage_df, valid_fullpage_df, test_ensemble_df,
                                       text_cols = 'text', label_cols = 'label', tokenizer=cust_tok, 
                                       vocab=transformer_vocab, include_bos=False, include_eos=False, 
                                       pad_first=False, pad_idx=pad_idx, bs=bs, num_workers=1)

In [ ]:
(acc, acc_with_prior), (f1, f1_with_prior) = eval_models.calcAccuracy_fullpage(learner, bpe_path, train_fullpage_df, valid_fullpage_df, test_ensemble_df, databunch=data_clas_test, ensembled=True)
(acc, acc_with_prior), (f1, f1_with_prior)

### Error Analysis

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12))